OutOfMemory<br>
<br>
論文<br>
https://arxiv.org/abs/2305.07015<br>
<br>
GitHub<br>
https://github.com/IceClear/StableSR<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/StableSR_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup environment

## git clone

In [ ]:
%cd /content

!git clone https://github.com/IceClear/StableSR.git

%cd /content/StableSR
# Commits on Jul 31, 2023
!git checkout 4796ae29e2ab80bd34873ec781ea69901292a122

## install libraries

In [ ]:
%cd /content/StableSR

# change pytorch version
!pip uninstall torch torchaudio torchdata torchmetrics torchsummary torch torchtext torchvision -y
!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 torchtext torchdata torchmetrics torchsummary xformers --extra-index-url https://download.pytorch.org/whl/cu116

# install taming & clip
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -e .

# install other library
!pip install omegaconf
!pip install einops
!pip install opencv-python
!pip install scipy
!pip install scikit-image
!pip install matplotlib
!pip install scikit-learn
!pip install transformers
!pip install kornia
!pip install open_clip_torch==2.0.2
!pip install pip install -U xformers
!pip install torchmetrics==0.6.0
!pip install pytorch-lightning==1.4.2

# download pretrain models

In [ ]:
%cd /content/StableSR

# Download the pre-trained model
!wget https://huggingface.co/Iceclear/StableSR/resolve/main/stablesr_000117.ckpt
!wget https://huggingface.co/Iceclear/StableSR/resolve/main/vqgan_cfw_00011.ckpt

# download input image

In [ ]:
%cd /content/StableSR

!mkdir -p inputs/downloads

!wget -c https://pbs.twimg.com/profile_images/702543084668776448/CXqtYbNg_400x400.jpg \
      -O ./inputs/downloads/test01.jpg

# Setup config

In [ ]:
#@markdown `DDPM_STEPS`: Number of DDPM steps for sampling<br>
DDPM_STEPS = 200 #@param {type:"slider", min:10, max:1000, step:10}
#@markdown `FIDELITY_WEIGHT`: Balance the quality (lower number) and fidelity (higher number)<br>
FIDELITY_WEIGHT = 0.97 #@param {type:"slider", min:0, max:1, step:0.01}
#@markdown `UPSCALE`: The upscale for super-resolution, 4x SR by default<br>
UPSCALE = 4.0 #@param {type:"slider", min:1.0, max:16.0, step:0.5}
#@markdown `SEED`: The random seed for sampling<br>
SEED = 42 #@param {type:"slider", min:0, max:10000, step:1}
#@markdown `TILE_OVERLAP`: The overlap between tiles, betwwen 0 to 64<br>
TILE_OVERLAP = 32 #@param {type:"slider", min:0, max:60, step:2}
#@markdown `VQGANTILE_SIZE`: The size for VQGAN tile operation in pixel, min 512.<br>
VQGANTILE_SIZE = 1280 #@param {type:"slider", min:512, max:2000, step:2}

VQGANTILE_STRIDE = int(VQGANTILE_SIZE * 0.9)

# Super Resolution

In [ ]:
!python scripts/sr_val_ddpm_text_T_vqganfin_old.py \
  --config configs/stableSRNew/v2-finetune_text_T_512.yaml \
  --ckpt './stablesr_000117.ckpt' \
  --vqgan_ckpt './vqgan_cfw_00011.ckpt' \
  --init-img 'inputs/downloads' \
  --outdir 'outputs/downloads' \
  --ddpm_steps {DDPM_STEPS} \
  --dec_w {FIDELITY_WEIGHT} \
  --seed {SEED} \
  --colorfix_type 'adain'